 LDA Latent Dirichlet Allocation

특정 토픽에 특정 단어가 나타날 확률을 알려주는 모형. 토픽별로 단어의 분포나 , 문서별로 토픽의 분포 추정 가능.



TF-IDF = 단어의 수를 세는 방식  

문서의 길이에 따라 수를 조정하기도 함.  
boolean 빈도로, 등장만 하면 1로 하기도 함.  
log로 조정하기도 함.  

TF * IDF 값으로. 그 문서에 얼마나 많이 나오는가, 다른 문서에는 얼마나 안나오는 단어인가.  

TF는 단어가 문서에 등장하는 횟수로, 같은 단어라도 문서에 따라 다르게 매겨진다.  
그러나 DF는 단어가 몇개의 문서에 등장했는지 나타내는 수로, 다른 문서라도 같은 값을 가지게 된다.  
IDF는  log(전체 문서수/해당 단어의 DF)를 의미.

즉, 전체 문서 뭉치에서 한 단어가 특정 문서에서만 많이 나오면 중요한 단어로 인식하게 하는 것.

$$tf-idf = tf * \log({ N \over{df}})$$

------------------
the bell이 2007년 등록, 그리고 다양한 신문에서 발생하는 기사들을 얻기 위해 다음 경제뉴스를 이용.

In [2]:
from newspaper import Article
import urllib.request
from bs4 import BeautifulSoup
import ssl
import datetime
import re
import pandas as pd

context = ssl._create_unverified_context()
context
# 클리닝 함수
def clean_text(text):
    text = text.replace("\n", " ")
    cleaned_text = re.sub('[a-zA-Z]', ' ', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                          ' ', cleaned_text)
    return cleaned_text

특정날짜에 해당하는 기사를 크롤링할 링크들을 크롤링.

In [30]:
url_1='https://media.daum.net/breakingnews/economic?regDate='
date=range(20131005,20131012)
#20131101 20131108  20131005 20131012    
url_2='&page='
page= range(1,20)

url_list=[]
for i in date:
    for j in page:
        mainurl=url_1 +str(i)+ url_2 + str(j) 
        source_code_from_URL = urllib.request.urlopen(mainurl, context=context)
        soup = BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8')
        list_news = soup.find('ul', attrs={'class':'list_news2'})
        list_news_li = list_news.find_all('li')
        for item in list_news_li:
            link_txt = item.find('a', attrs={'class':'link_txt'})
            url_list.append(link_txt.get('href'))


해당 링크에 대해 단어들을 수집하고 tf idf를 계산하여 저장

In [5]:
#크롤링1
from konlpy.tag import Kkma, Twitter, Komoran
from sklearn.feature_extraction.text import TfidfVectorizer
#twitter=Twitter()
kkma=Kkma()


mydoclist_kkma=[]


for url in url_list:
    article=Article(url, language='ko')
    article.download()
    article.parse()
    hoho=article.title+article.text
    
    kkma_nouns = ' '.join(kkma.nouns(hoho))
    mydoclist_kkma.append(kkma_nouns)

tfidf_vectorizer = TfidfVectorizer(min_df=1)
tfidf_vectorizer.fit(mydoclist_kkma)
tfidf_matrix_kkma =tfidf_vectorizer.transform(mydoclist_kkma)

#document_distances_kkma = tfidf_matrix_kkma * tfidf_matrix_kkma.T


내림차순으로 저장해서 tf idf가 높은 것부터 정렬시킴

In [195]:
import csv
sums = tfidf_matrix_kkma.sum(axis=0)

# connecting term to its sums frequency
data = []
for col, term in enumerate(terms):
    data.append( (term, sums[0,col] ))

ranking = pd.DataFrame(data, columns=['term','rank'])
ranking2=ranking.sort_values('rank', ascending=False)
ranking2.to_csv('tfidf.csv', encoding='ms949')


--------------

In [31]:
#크롤링2
from konlpy.tag import Kkma, Twitter, Komoran
from sklearn.feature_extraction.text import TfidfVectorizer
#twitter=Twitter()
kkma=Kkma()


mydoclist_kkma2=[]


for url in url_list:
    article=Article(url, language='ko')
    article.download()
    article.parse()
    hoho=article.title+article.text
    
    kkma_nouns2 = ' '.join(kkma.nouns(hoho))
    mydoclist_kkma2.append(kkma_nouns2)

tfidf_vectorizer2 = TfidfVectorizer(min_df=1)
tfidf_vectorizer2.fit(mydoclist_kkma2)
tfidf_matrix_kkma2 =tfidf_vectorizer2.transform(mydoclist_kkma2)

#document_distances_kkma = tfidf_matrix_kkma * tfidf_matrix_kkma.T

이번에는 금융 상태를 전반적으로 나타내는 단어를 찾기 위해 df값을 이용해서 벡터를 나타내기로 함, 그리고 그 두 벡터를 concate해서 코사인 유사도를 구하고자함.

In [128]:
aa={}
for i in range(len(list(tfidf_vectorizer2.vocabulary_.keys()))):
    aa[list(tfidf_vectorizer2.vocabulary_.keys())[i]]=tfidf_vectorizer2.idf_[i]


In [129]:
bb={}
for i in range(len(list(tfidf_vectorizer.vocabulary_.keys()))):
    bb[list(tfidf_vectorizer.vocabulary_.keys())[i]]=tfidf_vectorizer.idf_[i]


In [185]:
cc={}
dd={}
for i in aa.keys():
    cc[i]=0
    dd[i]=0
for j in bb.keys():
    cc[j]=0
    dd[j]=0

In [186]:
for i in aa.keys():
    cc[i]=aa[i]
for j in bb.keys():
    dd[j]=bb[j]

In [156]:
ccc=list(cc.values())
ddd=list(dd.values())

In [171]:
for i in range(65326):
    ccc[i]=2100/exp(ccc[i]-1)
    ddd[i]=2100/exp(ddd[i]-1)

In [178]:
from scipy import linalg, mat, dot
mc=mat(ccc)
md=mat(ddd)
sim_cd=dot(mc,md.T) / (linalg.norm(mc) * linalg.norm(md))
sim_cd

matrix([[0.0025001]])

----------------------

http://hero4earth.com/blog/projects/2018/01/21/naver_movie_review/

In [206]:
len(cc)

65326

군집 시작일 전 10일간의 기사를 기준으로 분포 확인.


doc2vec = 단어의 벡터 거리를 기준으로 유사도를 확인하는 방식


토픽 모델링을 이용한 뉴스기사 품질 평가 기사에대한 meta data 형성 후, 기사를 군집화 분류 / 뉴스 기사 제목으로 토픽 모델링. / 토픽 단어 분포와, 기사의 토픽 분포로 연관성score 도출 / 주어진 키워드에 대해 토픽 분포와 이질적으로 나타나는 기사를 선정(고유성 높음) - PCA기반, 가우시안 밀도 추정, knn, local outlier factors 